In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/humob2024/POIdata_cityD.csv
/kaggle/input/humob2024/sapporo_challengedata.csv
/kaggle/input/humob2024/POIdata_cityA.csv
/kaggle/input/humob2024/POI_datacategories.csv
/kaggle/input/humob2024/kumamoto_challengedata.csv
/kaggle/input/humob2024/POIdata_cityC.csv
/kaggle/input/humob2024/POIdata_cityB.csv
/kaggle/input/humob2024/hiroshima_challengedata.csv
/kaggle/input/humob2024/cityC_challengedata.csv/sapporo_challengedata.csv
/kaggle/input/humob2024/cityD_challengedata.csv/kumamoto_challengedata.csv
/kaggle/input/humob2024/cityA_groundtruthdata.csv/task1_dataset_kotae.csv
/kaggle/input/humob2024/cityB_challengedata.csv/hiroshima_challengedata.csv


In [2]:
# BLEUのインストール
!git clone https://github.com/yahoojapan/geobleu.git
!pip install /kaggle/working/geobleu/

Cloning into 'geobleu'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 156 (delta 75), reused 126 (delta 46), pack-reused 0 (from 0)
Receiving objects: 100% (156/156), 26.72 KiB | 855.00 KiB/s, done.
Resolving deltas: 100% (75/75), done.
Processing ./geobleu
  Preparing metadata (setup.py) ... done
  Created wheel for geobleu: filename=geobleu-0.3-py3-none-any.whl size=5037 sha256=2bee62c52aff4c9a30b9a02e8ce4a849189f5d49873486db2b6fe0e5067321c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-x3jsj_xi/wheels/ed/5d/64/4ba68bf198dc931381fb27d7eec45b1a2de54ca3eec437763a
Successfully built geobleu


In [ ]:
import geobleu

In [3]:
df_kotae = pl.read_csv("/kaggle/input/humob2024/cityA_groundtruthdata.csv/task1_dataset_kotae.csv")
len(df_kotae),df_kotae.head()

(111535175,
 shape: (5, 5)
 ┌─────┬─────┬─────┬─────┬─────┐
 │ uid ┆ d   ┆ t   ┆ x   ┆ y   │
 │ --- ┆ --- ┆ --- ┆ --- ┆ --- │
 │ i64 ┆ i64 ┆ i64 ┆ i64 ┆ i64 │
 ╞═════╪═════╪═════╪═════╪═════╡
 │ 0   ┆ 0   ┆ 1   ┆ 79  ┆ 86  │
 │ 0   ┆ 0   ┆ 2   ┆ 79  ┆ 86  │
 │ 0   ┆ 0   ┆ 8   ┆ 77  ┆ 86  │
 │ 0   ┆ 0   ┆ 9   ┆ 77  ┆ 86  │
 │ 0   ┆ 0   ┆ 19  ┆ 81  ┆ 89  │
 └─────┴─────┴─────┴─────┴─────┘)

In [4]:
print(df_kotae.select("d").min()),print(df_kotae.select("d").max())
print(df_kotae.select("x").max())

shape: (1, 1)
┌─────┐
│ d   │
│ --- │
│ i64 │
╞═════╡
│ 0   │
└─────┘
shape: (1, 1)
┌─────┐
│ d   │
│ --- │
│ i64 │
╞═════╡
│ 74  │
└─────┘
shape: (1, 1)
┌─────┐
│ x   │
│ --- │
│ i64 │
╞═════╡
│ 200 │
└─────┘


In [6]:
# validation対象のユーザーをサンプリング
num_uers = 5000
val_user = df_kotae.select("uid").unique().sample(num_uers,seed = 12).sort("uid").get_columns()[0].to_list()
print(val_user[:5])

[879, 1796, 3600, 3663, 5831]


In [7]:
# 60日目前後で訓練と検証に分ける
df_kotae_train =  df_kotae.filter(pl.col("d")<=60).filter(pl.col("uid").is_in(val_user))
df_kotae_test =  df_kotae.filter(pl.col("d")>=61).filter(pl.col("uid").is_in(val_user))

In [23]:
# 過去の同じ曜日のx,yの中央値を使う
xy_pred = []
for uid in val_user:
# for uid in [879]:
    uid_xy_pred = []
    
    for time_dur in range(0,15*48):
        day = time_dur//48+61
        time = time_dur%48
        # 同じ曜日、時間帯を抽出
        days = [d for d in range(day,0,-7)]
        df_kotae_train_weekdays = (df_kotae_train
                                   .filter(pl.col("uid")==uid)
                                   .filter(pl.col("d").is_in(days))
                                   .filter(pl.col("t")==time)
                                  )
        if len(df_kotae_train_weekdays)>=1:
            # 過去の同じ曜日、時間のログがある場合
            # 中央値を採用
#             _x = int(df_kotae_train_weekdays.select("x","y").median()[0,0])
#             _y = int(df_kotae_train_weekdays.select("x","y").median()[0,1])
            # 最頻値を採用
            _x = int(df_kotae_train_weekdays.group_by("x","y").count().sort("count", descending=True).select("x")[0,0])
            _y = int(df_kotae_train_weekdays.group_by("x","y").count().sort("count", descending=True).select("y")[0,0])
        # ない場合はひとつ前の予測を使う
        uid_xy_pred.append([uid,day,time,_x,_y])
    xy_pred.extend(uid_xy_pred)
xy_pred_array = np.array(xy_pred)
xy_pred_array[:5]

/tmp/ipykernel_36/490905812.py:22: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  _x = int(df_kotae_train_weekdays.group_by("x","y").count().sort("count", descending=True).select("x")[0,0])
/tmp/ipykernel_36/490905812.py:23: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  _y = int(df_kotae_train_weekdays.group_by("x","y").count().sort("count", descending=True).select("y")[0,0])


array([[879,  61,   0,  89,  14],
       [879,  61,   1,  88,  16],
       [879,  61,   2,  89,  16],
       [879,  61,   3,  88,  16],
       [879,  61,   4,  89,  14]])

In [24]:
# 検証データに予測データを結合
pred_dic = {
    'uid': xy_pred_array[:,0],
    'd': xy_pred_array[:,1],
    't': xy_pred_array[:,2],
    'x_pred': xy_pred_array[:,3],
    'y_pred': xy_pred_array[:,4],
}

df_pred = pl.DataFrame(pred_dic)
df_kotae_test_join_pred = df_kotae_test.join(df_pred,on=["uid","d","t"],how="left")
df_kotae_test_join_pred

uid,d,t,x,y,x_pred,y_pred
i64,i64,i64,i64,i64,i64,i64
879,61,5,89,14,88,16
879,61,8,88,16,89,15
879,61,9,89,14,89,14
879,61,13,89,15,89,14
879,61,14,89,15,90,22
…,…,…,…,…,…,…
98117,74,29,85,7,87,8
98117,74,30,87,8,88,8
98117,74,33,87,8,87,7


In [38]:
# # 一気に指標を計算　非常に時間がかかる
# generated = df_kotae_test_join_pred.select("uid","d", "t", "x", "y").to_numpy()
# generated = [tuple(ar) for ar in generated]
# reference = df_kotae_test_join_pred.select("uid","d", "t", "x_pred", "y_pred").to_numpy()
# reference = [tuple(ar) for ar in reference]
# geobleu_val = geobleu.calc_geobleu(generated, reference, processes=3)
# dtw_val = geobleu.calc_dtw(generated, reference, processes=3)

# geobleu_val,dtw_val

In [41]:
# ユーザーごとにgeobleuとdtwを計算して平均値をとる
geobleu_sum = 0
dtw_sum = 0
for uid in val_user:
    generated = df_kotae_test_join_pred.filter(pl.col("uid")==uid).select("d", "t", "x", "y").to_numpy()
    generated = [tuple(ar) for ar in generated]
    reference = df_kotae_test_join_pred.filter(pl.col("uid")==uid).select("d", "t", "x_pred", "y_pred").to_numpy()
    reference = [tuple(ar) for ar in reference]

    geobleu_val = geobleu.calc_geobleu(generated, reference, processes=4)
    print("geobleu: {}".format(geobleu_val))
    geobleu_sum += geobleu_val

    dtw_val = geobleu.calc_dtw(generated, reference, processes=4)
    print("dtw: {}".format(dtw_val))
    dtw_sum += dtw_val
print("avg geobleu: {}".format(geobleu_sum/len(val_user)))
print("avg dtw: {}".format(dtw_sum/len(val_user)))


geobleu: 0.5112480636245467
dtw: 17.86109871718166
geobleu: 0.17341169173588797
dtw: 38.249410313062164
geobleu: 0.007095018941575352
dtw: 143.71130373040063
geobleu: 0.38863480984309934
dtw: 23.88318941560463
geobleu: 0.5310876342194308
dtw: 39.135875220670655
geobleu: 0.01572392034976532
dtw: 126.1926747555376
geobleu: 0.3234451290408901
dtw: 22.481276404318823
geobleu: 0.026970340429041933
dtw: 119.77955935749839
geobleu: 0.1712786955276629
dtw: 54.70767237175788
geobleu: 0.20228081812588689
dtw: 20.196316067921618
geobleu: 0.24149637057386913
dtw: 31.060036442137424
geobleu: 0.21138277113902285
dtw: 102.7629496750883
geobleu: 0.4575467705256666
dtw: 18.903842928673647
geobleu: 0.1631313649992209
dtw: 29.010128290340877
geobleu: 0.4299808322776184
dtw: 12.882751807028104
geobleu: 0.14934375164742414
dtw: 86.89893532317893
geobleu: 0.08347590771635005
dtw: 33.998837626436014
geobleu: 0.5133892250520874
dtw: 177.17674550899548
geobleu: 0.22633535749184472
dtw: 26.652830105731024
geobl

In [ ]:
## 結果 ##　結構ぶれる
# 中央値の時
# n = 100
# avg geobleu: 0.16617719869411068
# avg dtw: 61.737381286226366
# n = 5000
# avg geobleu: 0.26376202174395486
# avg dtw: 55.09431442980051

# 最頻値の時
# n = 100
# avg geobleu: 0.2693492589666466
# avg dtw: 54.90577922313406
# n = 5000
# avg geobleu: 0.2631970725444824
# avg dtw: 57.8956081940585

In [17]:
# # 欠損を直前の座標で埋める処理
# # 全ての `t` の値を補完する関数を定義
# def fill_missing_t(df):
#     # 全ての `t` の範囲を作成
#     full_range = pl.DataFrame({'t': range(0,47)})

#     # 元のデータフレームを full_range と結合して、欠損を探す
#     df = full_range.join(df, on='t', how='left').sort(by=["uid","d","t"])

#     # 欠損している部分を直前のデータで埋める
#     df["","",""] = df.fill_null(strategy='forward')

#     return df

In [19]:
# def fill_xy(t_list, x_list, y_list):
#     res = [] # [(t,x,y),...]
#     x_ = 0
#     y_ = 0
#     t_ = 0
#     for t,x,y in zip(t_list, x_list, y_list):
#         if t != t_ + 1:
#             for t_i in range(t_+1, t):
#                 res.append((t_i,x_,y_))
#         x_ = x
#         y_ = y
#         t_ = t
#         res.append((t,x,y))
#     return res

In [ ]:
# tmp = df_sapporo.group_by("uid", "d").agg(pl.col("t"),pl.col("x"),pl.col("y")).sort("d")
# tmp

In [20]:
# tmp_ = (
#     tmp.with_columns(pl.struct("x", "t", "y")
#                      .map_elements(lambda x: fill_xy(x["t"], x["x"], x["y"])).alias("val"))
#     .select("uid", "d", "val")
#     .explode("val")
#     .with_columns(pl.col("val").list.get(0).alias("t"))
#     .with_columns(pl.col("val").list.get(1).alias("x"))
#     .with_columns(pl.col("val").list.get(2).alias("y"))
#     .drop("val")
# )
# tmp_